In [6]:
from bs4 import BeautifulSoup
import requests
import os

import threading
from multiprocessing import Process

savePath = r'C:/ESWork/ESPython/final_project/data/naver_data'


def getMovieCodeByYear(year):
    movieCode = []

    url = f'https://movie.naver.com/movie/sdb/browsing/bmovie.naver?open={year}&page=10000'
    response = requests.get(url)

    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'lxml')

        # html에서 div 태그 중 class이름이 pagenavigation 태그를 찾아라
        pagenavigation = soup.find('div', 'pagenavigation')
        # pagenavigation내 <a> 태그 중 마지막 태그의 text 값
        lastPage = pagenavigation.find_all('a')[-1].text

        for i in range(1, int(lastPage) + 1):
            url = f'https://movie.naver.com/movie/sdb/browsing/bmovie.naver?open={year}&page={i}'
            response2 = requests.get(url)

            if response2.status_code == 200:
                html = response2.text
                soup = BeautifulSoup(html, 'lxml')

                # html에서 ul 태그 중 class이름이 directory_list 태그를 찾아라
                directory_list = soup.find('ul', 'directory_list')
                allA = directory_list.findAll('a')
                for a in allA:
                    if '?code=' in str(a):
                        movieCode.append(str(a).split('?code=')[1].split('"')[0])
            else:
                print(response2.status_code)
    else:
        print(response.status_code)

    return year, movieCode


def getMovieInfo(year_codes):
    global savePath
    year = year_codes[0]
    codes = year_codes[1]
    
    delim = '\t'  # 각 요소 별 구분자, 콤마(,)는 제목,장르 등에 쓰이기 때문에 탭(\t)으로 구분.
    f = open(os.path.join(savePath, f'{year}.csv'), 'w', encoding='UTF-8')
    f.write('openYear' + delim)
    f.write('code' + delim)
    f.write('url' + delim)
    f.write('title' + delim)
    f.write('title_forn' + delim)
    f.write('poster_url' + delim)
    f.write('genre' + delim)
    f.write('nation' + delim)
    f.write('runningTime' + delim)
    f.write('age' + delim)
    f.write('openDate' + delim)
    f.write('netizens_rate' + delim)
    f.write('netizens_participate' + delim)
    f.write('critics_rate' + delim)
    f.write('critics_participate' + delim)
    f.write('directors' + delim)
    f.write('actors' + delim)
    f.write('story' + delim + '\n')

    # t = threading.Thread(target=getImage, args=(os.path.join(savePath, str(year)), codes))
    # t.start()

    for i, code in enumerate(codes):
        # print(f'{i}/{len(codes)}', end='\r')
        url = f'https://movie.naver.com/movie/bi/mi/basic.naver?code={code}'
        response = requests.get(url)

        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'lxml')

            title = soup.find('head').find('title').text.replace(' : 네이버 영화', '')
            
            if title != '' :
                title_forn_ing = soup.find('div', class_='mv_info_area').find('strong', class_='h_movie2').text.find(',')
                if title_forn_ing == -1:
                    title_forn = ''
                else:
                    title_forn = soup.find('div', class_='mv_info_area').find('strong', class_='h_movie2').text.split(',')[-2].strip()

                poster_ing = soup.find('div', class_='mv_info_area').find('div', class_='poster').find('img')
                if poster_ing == -1:
                    poster_url = ''
                else:
                    poster_url = poster_ing['src']
                
                netizens_rate_ing = soup.find('div', class_='netizen_score').find('div', class_='star_score')
                if netizens_rate_ing == -1 :
                    netizens_rate = ''
                else:
                    netizens_rate = netizens_rate_ing.find('em').get_text()
                       
                netizens_participate_ing = soup.find('div', class_='netizen_score').find('span', class_='user_count')
                if netizens_participate_ing == -1:
                    netizens_participate = ''
                else:
                    netizens_participate = netizens_participate_ing.find('em').get_text()
                    
                critics_rate_ing = soup.find('div', class_='special_score').find('div', class_='star_score')
                if critics_rate_ing == -1 :
                    critics_rate = ''
                else:
                    critics_rate = critics_rate_ing.find('em').get_text()

                critics_participate_ing = soup.find('div', class_='special_score').find('span', class_='user_count')
                if critics_participate_ing == -1:
                    critics_participate = ''
                else:
                    critics_participate = critics_participate_ing.find('em').get_text()
                
                                
            else:
                title_forn = ''
                poster_url = ''
                netizens_rate = ''
                netizens_participate = ''
                critics_rate = ''
                critics_participate = ''

#             netizens_rate, netizens_participate = getNetizensRate(soup.find_all('div', 'title_area grade_tit'))
#             critics_rate, critics_participate = getCriticsRate(soup.find_all('div', 'score_special'))

            info_spec = soup.find('dl', 'info_spec')

            f.write(str(year) + delim)  # year
            f.write(code + delim)  # code
            f.write(url + delim)  # url
            f.write(title + delim)  # title
            f.write(title_forn + delim)  # title_forn
            f.write(poster_url + delim)  # poster_url
            print(url)

            if info_spec is not None:
                info_spec = getInfoSpec(info_spec)
                f.write(info_spec['장르'] + delim)  # genre
                f.write(info_spec['국가'] + delim)  # nation
                f.write(info_spec['상영시간'] + delim)  # runningTime
                f.write(info_spec['등급'] + delim)  # age
                f.write(info_spec['개봉'] + delim)  # openDate

                # 네티즌 평점
                if netizens_rate:
                    f.write(netizens_rate + delim + netizens_participate + delim)
                else:
                    f.write(delim + delim)

                # 전문가 평점
                if critics_rate:
                    f.write(critics_rate + delim + critics_participate + delim)
                else:
                    f.write(delim + delim)

                f.write(info_spec['감독'] + delim)  # directors
                f.write(info_spec['출연'] + delim)  # actors

            story = getStory(f'https://movie.naver.com/movie/bi/mi/basic.naver?code={code}')
            if story:
                f.write(story)

            f.write('\n')

        else:
            print(response.status_code)

    f.close()


def getNetizensRate(htmls):
    rate = ''
    participate = ''
    for html in htmls:
        if html.find('a', id='netizen_point_tab') is None:
            continue
        if html.find('a', id='netizen_point_tab').text == '네티즌 평점':
            for val in html.find('div', 'star_score').find_all('em'):
                rate += val.text
            if rate != '':
                participate = html.find('span', 'user_count').find('em').text
                return rate, participate
    return False, False

def getCriticsRate(htmls):
    rate = ''
    participate = ''
    for html in htmls:
        if html.find('strong', 'blind') is None:
            continue
        if html.find('strong', 'blind').text == '기자 · 평론가 평점':
            for val in html.find('div', 'star_score').find_all('em'):
                rate += val.text
            if rate != '':
                participate = html.find('span', 'user_count').find('em').text
                return rate, participate

    return False, False


def getStory(url):
    response = requests.get(url)

    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'lxml')
        story = soup.find('p', 'con_tx')
        if story is not None:
            story = str(story).replace('<p class="con_tx">', '').replace('</p>', '').replace('\n', '').replace('\t', '')
            # <br/>뒤에 있는 공백은 ' '이 아니라 '\xa0'이다.
            # 공백이 html에서 &nbsp;로 표현되서 그렇나 봄?
            story = story.replace('&lt;', '<').replace('&gt;', '>').replace('\r', '').replace('\xa0', '')

            return story
        return False
    else:
        print(response.status_code)


def getImage(path, codes):
    if not os.path.isdir(os.path.join(path)):
        os.makedirs(path)

    for code in codes:
        url = f'https://movie.naver.com/movie/bi/mi/photoViewPopup.naver?movieCode={code}'
        response = requests.get(url)

        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'lxml')
            imgUrl = soup.find('img', id='targetImage')
            if imgUrl is not None:
                imgUrl = imgUrl.attrs['src']
                with open(os.path.join(path, code + '.png'), 'wb') as poster:
                    poster.write(requests.get(imgUrl).content)
        else:
            print(response.status_code)


def getInfoSpec(html):
    dtList = html.find_all('dt')
    ddList = html.find_all('dd')

    # 딕셔너리로 가져올 정보를 미리 정해 놓는다.
    info = {'장르': '',
            '상영시간': '',
            '국가': '',
            '개봉': '',
            '감독': '',
            '출연': '',
            '등급': ''}

    # dt, dd의 개수만큽
    for dt, dd in zip(dtList, ddList):
        if '개요' in dt.text:
            for span in dd.find_all('span'):
                if '분' in span.text:
                    info['상영시간'] = span.text
                else:
                    for a in span.find_all('a'):
                        if 'genre' in str(a):
                            # 공연실황 제외 - 이건 뭐 하는건지?
                            # 멜로/로맨스 - 이상한 19금 영화가 엄청 많음, 어떻게 거를 방법이...
                            if '공연실황' in a.text:
                                continue
                            if info['장르'] != '':
                                info['장르'] += ', ' + a.text
                            else:
                                info['장르'] = a.text
                        # 국가도 여러개인 경우가 있네요
                        elif 'nation' in str(a):
                            if info['국가'] != '':
                                info['국가'] += ', ' + a.text
                            else:
                                info['국가'] = a.text
                        # 재개봉 영화의 경우 개봉이 여러개로 들어온다.
                        elif 'open' in str(a):
                            if info['개봉'] != '':
                                info['개봉'] += a.text
                            else:
                                info['개봉'] = a.text
        elif '감독' in dt.text:
            for a in dd.find_all('a'):
                # 감독이 여러명인 경우가 있어서 - 루소 형제 등?
                if info['감독'] != '':
                    info['감독'] += ', ' + a.text
                else:
                    info['감독'] = a.text
        elif '출연' in dt.text:
            for a in dd.find_all('a'):
                # 메인 출연진이 아니라면 pass
                if '더보기' in a.text:
                    continue
                if info['출연'] != '':
                    info['출연'] += ', ' + a.text
                else:
                    info['출연'] = a.text
        # 등급의 경우 한국 등급만 가져오려면 수정이 필요
        # 한국 등급이 없는 경우를 대비해서 일본, 미국 등의 등급이라도 가져오기 위해
        # 별다른 수정은 안 했습니다.
        elif '등급' in dt.text:
            for a in dd.find_all('a'):
                if 'grade' in str(a):
                    if info['등급'] != '':
                        info['등급'] += ', ' + a.text
                    else:
                        info['등급'] = a.text
    return info


def crawling(s, e):
    for i in range(s, e + 1):
        getMovieInfo(getMovieCodeByYear(i))


# if __name__ == "__main__":
#     p1 = Process(target=crawling, args=(1950, 1980))
#     p2 = Process(target=crawling, args=(1981, 1990))
#     p3 = Process(target=crawling, args=(1991, 2000))
#     p4 = Process(target=crawling, args=(2001, 2010))
#     p5 = Process(target=crawling, args=(2011, 2015))
#     p6 = Process(target=crawling, args=(2016, 2022))

#     p1.start()
#     p2.start()
#     p3.start()
#     p4.start()
#     p5.start()
#     p6.start()

#     p1.join()
#     p2.join()
#     p3.join()
#     p4.join()
#     p5.join()
#     p6.join()

In [7]:
crawling(1950, 1980)

https://movie.naver.com/movie/bi/mi/basic.naver?code=49697
https://movie.naver.com/movie/bi/mi/basic.naver?code=147087
https://movie.naver.com/movie/bi/mi/basic.naver?code=21867
https://movie.naver.com/movie/bi/mi/basic.naver?code=22189
https://movie.naver.com/movie/bi/mi/basic.naver?code=13543
https://movie.naver.com/movie/bi/mi/basic.naver?code=23697
https://movie.naver.com/movie/bi/mi/basic.naver?code=23829
https://movie.naver.com/movie/bi/mi/basic.naver?code=10265
https://movie.naver.com/movie/bi/mi/basic.naver?code=22366
https://movie.naver.com/movie/bi/mi/basic.naver?code=19737
https://movie.naver.com/movie/bi/mi/basic.naver?code=19745
https://movie.naver.com/movie/bi/mi/basic.naver?code=19811
https://movie.naver.com/movie/bi/mi/basic.naver?code=19876
https://movie.naver.com/movie/bi/mi/basic.naver?code=19905
https://movie.naver.com/movie/bi/mi/basic.naver?code=19913
https://movie.naver.com/movie/bi/mi/basic.naver?code=19932
https://movie.naver.com/movie/bi/mi/basic.naver?code=19

https://movie.naver.com/movie/bi/mi/basic.naver?code=20228
https://movie.naver.com/movie/bi/mi/basic.naver?code=20235
https://movie.naver.com/movie/bi/mi/basic.naver?code=20265
https://movie.naver.com/movie/bi/mi/basic.naver?code=20279
https://movie.naver.com/movie/bi/mi/basic.naver?code=20271
https://movie.naver.com/movie/bi/mi/basic.naver?code=20432
https://movie.naver.com/movie/bi/mi/basic.naver?code=20440
https://movie.naver.com/movie/bi/mi/basic.naver?code=20441
https://movie.naver.com/movie/bi/mi/basic.naver?code=20443
https://movie.naver.com/movie/bi/mi/basic.naver?code=20451
https://movie.naver.com/movie/bi/mi/basic.naver?code=20486
https://movie.naver.com/movie/bi/mi/basic.naver?code=33275
https://movie.naver.com/movie/bi/mi/basic.naver?code=20512
https://movie.naver.com/movie/bi/mi/basic.naver?code=20526
https://movie.naver.com/movie/bi/mi/basic.naver?code=20531
https://movie.naver.com/movie/bi/mi/basic.naver?code=20532
https://movie.naver.com/movie/bi/mi/basic.naver?code=205

https://movie.naver.com/movie/bi/mi/basic.naver?code=22530
https://movie.naver.com/movie/bi/mi/basic.naver?code=22540
https://movie.naver.com/movie/bi/mi/basic.naver?code=22510
https://movie.naver.com/movie/bi/mi/basic.naver?code=22549
https://movie.naver.com/movie/bi/mi/basic.naver?code=22578
https://movie.naver.com/movie/bi/mi/basic.naver?code=22580
https://movie.naver.com/movie/bi/mi/basic.naver?code=22617
https://movie.naver.com/movie/bi/mi/basic.naver?code=22618
https://movie.naver.com/movie/bi/mi/basic.naver?code=22642
https://movie.naver.com/movie/bi/mi/basic.naver?code=22659
https://movie.naver.com/movie/bi/mi/basic.naver?code=22660
https://movie.naver.com/movie/bi/mi/basic.naver?code=22676
https://movie.naver.com/movie/bi/mi/basic.naver?code=22687
https://movie.naver.com/movie/bi/mi/basic.naver?code=22733
https://movie.naver.com/movie/bi/mi/basic.naver?code=22740
https://movie.naver.com/movie/bi/mi/basic.naver?code=22743
https://movie.naver.com/movie/bi/mi/basic.naver?code=227

https://movie.naver.com/movie/bi/mi/basic.naver?code=62441
https://movie.naver.com/movie/bi/mi/basic.naver?code=21914
https://movie.naver.com/movie/bi/mi/basic.naver?code=15785
https://movie.naver.com/movie/bi/mi/basic.naver?code=22016
https://movie.naver.com/movie/bi/mi/basic.naver?code=16286
https://movie.naver.com/movie/bi/mi/basic.naver?code=22225
https://movie.naver.com/movie/bi/mi/basic.naver?code=22230
https://movie.naver.com/movie/bi/mi/basic.naver?code=22235
https://movie.naver.com/movie/bi/mi/basic.naver?code=22280
https://movie.naver.com/movie/bi/mi/basic.naver?code=22310
https://movie.naver.com/movie/bi/mi/basic.naver?code=13021
https://movie.naver.com/movie/bi/mi/basic.naver?code=22349
https://movie.naver.com/movie/bi/mi/basic.naver?code=22404
https://movie.naver.com/movie/bi/mi/basic.naver?code=15859
https://movie.naver.com/movie/bi/mi/basic.naver?code=22473
https://movie.naver.com/movie/bi/mi/basic.naver?code=22475
https://movie.naver.com/movie/bi/mi/basic.naver?code=184

In [8]:
crawling(1981, 1990)

https://movie.naver.com/movie/bi/mi/basic.naver?code=12654
https://movie.naver.com/movie/bi/mi/basic.naver?code=10994
https://movie.naver.com/movie/bi/mi/basic.naver?code=10727
https://movie.naver.com/movie/bi/mi/basic.naver?code=10630
https://movie.naver.com/movie/bi/mi/basic.naver?code=10926
https://movie.naver.com/movie/bi/mi/basic.naver?code=14478
https://movie.naver.com/movie/bi/mi/basic.naver?code=14003
https://movie.naver.com/movie/bi/mi/basic.naver?code=62598
https://movie.naver.com/movie/bi/mi/basic.naver?code=10905
https://movie.naver.com/movie/bi/mi/basic.naver?code=19709
https://movie.naver.com/movie/bi/mi/basic.naver?code=62492
https://movie.naver.com/movie/bi/mi/basic.naver?code=62426
https://movie.naver.com/movie/bi/mi/basic.naver?code=19749
https://movie.naver.com/movie/bi/mi/basic.naver?code=13132
https://movie.naver.com/movie/bi/mi/basic.naver?code=10096
https://movie.naver.com/movie/bi/mi/basic.naver?code=28507
https://movie.naver.com/movie/bi/mi/basic.naver?code=157

https://movie.naver.com/movie/bi/mi/basic.naver?code=21189
https://movie.naver.com/movie/bi/mi/basic.naver?code=21193
https://movie.naver.com/movie/bi/mi/basic.naver?code=10519
https://movie.naver.com/movie/bi/mi/basic.naver?code=10371
https://movie.naver.com/movie/bi/mi/basic.naver?code=10521
https://movie.naver.com/movie/bi/mi/basic.naver?code=10971
https://movie.naver.com/movie/bi/mi/basic.naver?code=10502
https://movie.naver.com/movie/bi/mi/basic.naver?code=62506
https://movie.naver.com/movie/bi/mi/basic.naver?code=17646
https://movie.naver.com/movie/bi/mi/basic.naver?code=10564
https://movie.naver.com/movie/bi/mi/basic.naver?code=15384
https://movie.naver.com/movie/bi/mi/basic.naver?code=39608
https://movie.naver.com/movie/bi/mi/basic.naver?code=21269
https://movie.naver.com/movie/bi/mi/basic.naver?code=21274
https://movie.naver.com/movie/bi/mi/basic.naver?code=10003
https://movie.naver.com/movie/bi/mi/basic.naver?code=10004
https://movie.naver.com/movie/bi/mi/basic.naver?code=112

https://movie.naver.com/movie/bi/mi/basic.naver?code=11141
https://movie.naver.com/movie/bi/mi/basic.naver?code=13748
https://movie.naver.com/movie/bi/mi/basic.naver?code=13684
https://movie.naver.com/movie/bi/mi/basic.naver?code=14382
https://movie.naver.com/movie/bi/mi/basic.naver?code=13255
https://movie.naver.com/movie/bi/mi/basic.naver?code=23105
https://movie.naver.com/movie/bi/mi/basic.naver?code=17647
https://movie.naver.com/movie/bi/mi/basic.naver?code=23130
https://movie.naver.com/movie/bi/mi/basic.naver?code=10686
https://movie.naver.com/movie/bi/mi/basic.naver?code=11003
https://movie.naver.com/movie/bi/mi/basic.naver?code=10626
https://movie.naver.com/movie/bi/mi/basic.naver?code=12828
https://movie.naver.com/movie/bi/mi/basic.naver?code=13479
https://movie.naver.com/movie/bi/mi/basic.naver?code=10450
https://movie.naver.com/movie/bi/mi/basic.naver?code=13360
https://movie.naver.com/movie/bi/mi/basic.naver?code=13169
https://movie.naver.com/movie/bi/mi/basic.naver?code=232

In [9]:
crawling(1991, 2000)

https://movie.naver.com/movie/bi/mi/basic.naver?code=21800
https://movie.naver.com/movie/bi/mi/basic.naver?code=21806
https://movie.naver.com/movie/bi/mi/basic.naver?code=11509
https://movie.naver.com/movie/bi/mi/basic.naver?code=21200
https://movie.naver.com/movie/bi/mi/basic.naver?code=11087
https://movie.naver.com/movie/bi/mi/basic.naver?code=15064
https://movie.naver.com/movie/bi/mi/basic.naver?code=16709
https://movie.naver.com/movie/bi/mi/basic.naver?code=84252
https://movie.naver.com/movie/bi/mi/basic.naver?code=22131
https://movie.naver.com/movie/bi/mi/basic.naver?code=11096
https://movie.naver.com/movie/bi/mi/basic.naver?code=10936
https://movie.naver.com/movie/bi/mi/basic.naver?code=11337
https://movie.naver.com/movie/bi/mi/basic.naver?code=15399
https://movie.naver.com/movie/bi/mi/basic.naver?code=15506
https://movie.naver.com/movie/bi/mi/basic.naver?code=12504
https://movie.naver.com/movie/bi/mi/basic.naver?code=18279
https://movie.naver.com/movie/bi/mi/basic.naver?code=155

https://movie.naver.com/movie/bi/mi/basic.naver?code=11171
https://movie.naver.com/movie/bi/mi/basic.naver?code=10454
https://movie.naver.com/movie/bi/mi/basic.naver?code=11815
https://movie.naver.com/movie/bi/mi/basic.naver?code=10767
https://movie.naver.com/movie/bi/mi/basic.naver?code=90071
https://movie.naver.com/movie/bi/mi/basic.naver?code=75444
https://movie.naver.com/movie/bi/mi/basic.naver?code=35983
https://movie.naver.com/movie/bi/mi/basic.naver?code=15209
https://movie.naver.com/movie/bi/mi/basic.naver?code=10360
https://movie.naver.com/movie/bi/mi/basic.naver?code=11119
https://movie.naver.com/movie/bi/mi/basic.naver?code=19600
https://movie.naver.com/movie/bi/mi/basic.naver?code=11519
https://movie.naver.com/movie/bi/mi/basic.naver?code=11797
https://movie.naver.com/movie/bi/mi/basic.naver?code=13455
https://movie.naver.com/movie/bi/mi/basic.naver?code=13444
https://movie.naver.com/movie/bi/mi/basic.naver?code=17321
https://movie.naver.com/movie/bi/mi/basic.naver?code=106

https://movie.naver.com/movie/bi/mi/basic.naver?code=14262
https://movie.naver.com/movie/bi/mi/basic.naver?code=11869
https://movie.naver.com/movie/bi/mi/basic.naver?code=23410
https://movie.naver.com/movie/bi/mi/basic.naver?code=13567
https://movie.naver.com/movie/bi/mi/basic.naver?code=62509
https://movie.naver.com/movie/bi/mi/basic.naver?code=13321
https://movie.naver.com/movie/bi/mi/basic.naver?code=21748
https://movie.naver.com/movie/bi/mi/basic.naver?code=27095
https://movie.naver.com/movie/bi/mi/basic.naver?code=27788
https://movie.naver.com/movie/bi/mi/basic.naver?code=23453
https://movie.naver.com/movie/bi/mi/basic.naver?code=11088
https://movie.naver.com/movie/bi/mi/basic.naver?code=16252
https://movie.naver.com/movie/bi/mi/basic.naver?code=14333
https://movie.naver.com/movie/bi/mi/basic.naver?code=11677
https://movie.naver.com/movie/bi/mi/basic.naver?code=12045
https://movie.naver.com/movie/bi/mi/basic.naver?code=13597
https://movie.naver.com/movie/bi/mi/basic.naver?code=235

https://movie.naver.com/movie/bi/mi/basic.naver?code=10284
https://movie.naver.com/movie/bi/mi/basic.naver?code=11491
https://movie.naver.com/movie/bi/mi/basic.naver?code=12934
https://movie.naver.com/movie/bi/mi/basic.naver?code=13736
https://movie.naver.com/movie/bi/mi/basic.naver?code=16868
https://movie.naver.com/movie/bi/mi/basic.naver?code=11742
https://movie.naver.com/movie/bi/mi/basic.naver?code=11168
https://movie.naver.com/movie/bi/mi/basic.naver?code=90087
https://movie.naver.com/movie/bi/mi/basic.naver?code=17106
https://movie.naver.com/movie/bi/mi/basic.naver?code=12535
https://movie.naver.com/movie/bi/mi/basic.naver?code=16877
https://movie.naver.com/movie/bi/mi/basic.naver?code=10810
https://movie.naver.com/movie/bi/mi/basic.naver?code=11144
https://movie.naver.com/movie/bi/mi/basic.naver?code=11297
https://movie.naver.com/movie/bi/mi/basic.naver?code=11404
https://movie.naver.com/movie/bi/mi/basic.naver?code=13172
https://movie.naver.com/movie/bi/mi/basic.naver?code=115

https://movie.naver.com/movie/bi/mi/basic.naver?code=12195
https://movie.naver.com/movie/bi/mi/basic.naver?code=62740
https://movie.naver.com/movie/bi/mi/basic.naver?code=13747
https://movie.naver.com/movie/bi/mi/basic.naver?code=22248
https://movie.naver.com/movie/bi/mi/basic.naver?code=10741
https://movie.naver.com/movie/bi/mi/basic.naver?code=12330
https://movie.naver.com/movie/bi/mi/basic.naver?code=28196
https://movie.naver.com/movie/bi/mi/basic.naver?code=27858
https://movie.naver.com/movie/bi/mi/basic.naver?code=14158
https://movie.naver.com/movie/bi/mi/basic.naver?code=14159
https://movie.naver.com/movie/bi/mi/basic.naver?code=13055
https://movie.naver.com/movie/bi/mi/basic.naver?code=28340
https://movie.naver.com/movie/bi/mi/basic.naver?code=12596
https://movie.naver.com/movie/bi/mi/basic.naver?code=13221
https://movie.naver.com/movie/bi/mi/basic.naver?code=12363
https://movie.naver.com/movie/bi/mi/basic.naver?code=30904
https://movie.naver.com/movie/bi/mi/basic.naver?code=127

...

https://movie.naver.com/movie/bi/mi/basic.naver?code=27896
https://movie.naver.com/movie/bi/mi/basic.naver?code=90468
https://movie.naver.com/movie/bi/mi/basic.naver?code=25587
https://movie.naver.com/movie/bi/mi/basic.naver?code=26278
https://movie.naver.com/movie/bi/mi/basic.naver?code=25263
https://movie.naver.com/movie/bi/mi/basic.naver?code=19095
https://movie.naver.com/movie/bi/mi/basic.naver?code=25366
https://movie.naver.com/movie/bi/mi/basic.naver?code=38651
https://movie.naver.com/movie/bi/mi/basic.naver?code=23859
https://movie.naver.com/movie/bi/mi/basic.naver?code=22107
https://movie.naver.com/movie/bi/mi/basic.naver?code=25264
https://movie.naver.com/movie/bi/mi/basic.naver?code=24830
https://movie.naver.com/movie/bi/mi/basic.naver?code=23853
https://movie.naver.com/movie/bi/mi/basic.naver?code=19543
https://movie.naver.com/movie/bi/mi/basic.naver?code=19525
https://movie.naver.com/movie/bi/mi/basic.naver?code=25634
https://movie.naver.com/movie/bi/mi/basic.naver?code=260

https://movie.naver.com/movie/bi/mi/basic.naver?code=21147
https://movie.naver.com/movie/bi/mi/basic.naver?code=25332
https://movie.naver.com/movie/bi/mi/basic.naver?code=27292
https://movie.naver.com/movie/bi/mi/basic.naver?code=23850
https://movie.naver.com/movie/bi/mi/basic.naver?code=35781
https://movie.naver.com/movie/bi/mi/basic.naver?code=93625
https://movie.naver.com/movie/bi/mi/basic.naver?code=25404
https://movie.naver.com/movie/bi/mi/basic.naver?code=24194
https://movie.naver.com/movie/bi/mi/basic.naver?code=25392
https://movie.naver.com/movie/bi/mi/basic.naver?code=26021
https://movie.naver.com/movie/bi/mi/basic.naver?code=25181
https://movie.naver.com/movie/bi/mi/basic.naver?code=24484
https://movie.naver.com/movie/bi/mi/basic.naver?code=18994
https://movie.naver.com/movie/bi/mi/basic.naver?code=25639
https://movie.naver.com/movie/bi/mi/basic.naver?code=27633
https://movie.naver.com/movie/bi/mi/basic.naver?code=26020
https://movie.naver.com/movie/bi/mi/basic.naver?code=514

https://movie.naver.com/movie/bi/mi/basic.naver?code=39772
https://movie.naver.com/movie/bi/mi/basic.naver?code=26350
https://movie.naver.com/movie/bi/mi/basic.naver?code=39497
https://movie.naver.com/movie/bi/mi/basic.naver?code=39419
https://movie.naver.com/movie/bi/mi/basic.naver?code=29411
https://movie.naver.com/movie/bi/mi/basic.naver?code=25784
https://movie.naver.com/movie/bi/mi/basic.naver?code=25586
https://movie.naver.com/movie/bi/mi/basic.naver?code=24108
https://movie.naver.com/movie/bi/mi/basic.naver?code=23860
https://movie.naver.com/movie/bi/mi/basic.naver?code=27106
https://movie.naver.com/movie/bi/mi/basic.naver?code=24825
https://movie.naver.com/movie/bi/mi/basic.naver?code=29688
https://movie.naver.com/movie/bi/mi/basic.naver?code=29134
https://movie.naver.com/movie/bi/mi/basic.naver?code=24880
https://movie.naver.com/movie/bi/mi/basic.naver?code=31531
https://movie.naver.com/movie/bi/mi/basic.naver?code=31285
https://movie.naver.com/movie/bi/mi/basic.naver?code=244

https://movie.naver.com/movie/bi/mi/basic.naver?code=15891
https://movie.naver.com/movie/bi/mi/basic.naver?code=41366
https://movie.naver.com/movie/bi/mi/basic.naver?code=18763
https://movie.naver.com/movie/bi/mi/basic.naver?code=28788
https://movie.naver.com/movie/bi/mi/basic.naver?code=27479
https://movie.naver.com/movie/bi/mi/basic.naver?code=27219
https://movie.naver.com/movie/bi/mi/basic.naver?code=39500
https://movie.naver.com/movie/bi/mi/basic.naver?code=28845
https://movie.naver.com/movie/bi/mi/basic.naver?code=35279
https://movie.naver.com/movie/bi/mi/basic.naver?code=30413
https://movie.naver.com/movie/bi/mi/basic.naver?code=29083
https://movie.naver.com/movie/bi/mi/basic.naver?code=24674
https://movie.naver.com/movie/bi/mi/basic.naver?code=25641
https://movie.naver.com/movie/bi/mi/basic.naver?code=29568
https://movie.naver.com/movie/bi/mi/basic.naver?code=29054
https://movie.naver.com/movie/bi/mi/basic.naver?code=27770
https://movie.naver.com/movie/bi/mi/basic.naver?code=300

https://movie.naver.com/movie/bi/mi/basic.naver?code=31156
https://movie.naver.com/movie/bi/mi/basic.naver?code=30720
https://movie.naver.com/movie/bi/mi/basic.naver?code=27513
https://movie.naver.com/movie/bi/mi/basic.naver?code=15295
https://movie.naver.com/movie/bi/mi/basic.naver?code=29299
https://movie.naver.com/movie/bi/mi/basic.naver?code=26271
https://movie.naver.com/movie/bi/mi/basic.naver?code=26792
https://movie.naver.com/movie/bi/mi/basic.naver?code=29257
https://movie.naver.com/movie/bi/mi/basic.naver?code=27564
https://movie.naver.com/movie/bi/mi/basic.naver?code=26351
https://movie.naver.com/movie/bi/mi/basic.naver?code=29516
https://movie.naver.com/movie/bi/mi/basic.naver?code=38782
https://movie.naver.com/movie/bi/mi/basic.naver?code=29053
https://movie.naver.com/movie/bi/mi/basic.naver?code=28844
https://movie.naver.com/movie/bi/mi/basic.naver?code=29753
https://movie.naver.com/movie/bi/mi/basic.naver?code=30646
https://movie.naver.com/movie/bi/mi/basic.naver?code=299

In [6]:
crawling(2001, 2010)

https://movie.naver.com/movie/bi/mi/point.naver?code=31964
https://movie.naver.com/movie/bi/mi/point.naver?code=31595
https://movie.naver.com/movie/bi/mi/point.naver?code=36752
https://movie.naver.com/movie/bi/mi/point.naver?code=31897
https://movie.naver.com/movie/bi/mi/point.naver?code=31245
https://movie.naver.com/movie/bi/mi/point.naver?code=31171
https://movie.naver.com/movie/bi/mi/point.naver?code=30905
https://movie.naver.com/movie/bi/mi/point.naver?code=60250
https://movie.naver.com/movie/bi/mi/point.naver?code=84295
https://movie.naver.com/movie/bi/mi/point.naver?code=39203
https://movie.naver.com/movie/bi/mi/point.naver?code=33302
https://movie.naver.com/movie/bi/mi/point.naver?code=30748
https://movie.naver.com/movie/bi/mi/point.naver?code=27474
https://movie.naver.com/movie/bi/mi/point.naver?code=24051
https://movie.naver.com/movie/bi/mi/point.naver?code=32744
https://movie.naver.com/movie/bi/mi/point.naver?code=33712
https://movie.naver.com/movie/bi/mi/point.naver?code=335

https://movie.naver.com/movie/bi/mi/point.naver?code=32880
https://movie.naver.com/movie/bi/mi/point.naver?code=29505
https://movie.naver.com/movie/bi/mi/point.naver?code=41342
https://movie.naver.com/movie/bi/mi/point.naver?code=32132
https://movie.naver.com/movie/bi/mi/point.naver?code=38127
https://movie.naver.com/movie/bi/mi/point.naver?code=31200
https://movie.naver.com/movie/bi/mi/point.naver?code=31118
https://movie.naver.com/movie/bi/mi/point.naver?code=31596
https://movie.naver.com/movie/bi/mi/point.naver?code=31389
https://movie.naver.com/movie/bi/mi/point.naver?code=31349
https://movie.naver.com/movie/bi/mi/point.naver?code=32814
https://movie.naver.com/movie/bi/mi/point.naver?code=31583
https://movie.naver.com/movie/bi/mi/point.naver?code=32036
https://movie.naver.com/movie/bi/mi/point.naver?code=31809
https://movie.naver.com/movie/bi/mi/point.naver?code=57984
https://movie.naver.com/movie/bi/mi/point.naver?code=31799
https://movie.naver.com/movie/bi/mi/point.naver?code=330

https://movie.naver.com/movie/bi/mi/point.naver?code=29219
https://movie.naver.com/movie/bi/mi/point.naver?code=27307
https://movie.naver.com/movie/bi/mi/point.naver?code=34034
https://movie.naver.com/movie/bi/mi/point.naver?code=31781
https://movie.naver.com/movie/bi/mi/point.naver?code=30421
https://movie.naver.com/movie/bi/mi/point.naver?code=32038
https://movie.naver.com/movie/bi/mi/point.naver?code=30094
https://movie.naver.com/movie/bi/mi/point.naver?code=31272
https://movie.naver.com/movie/bi/mi/point.naver?code=31345
https://movie.naver.com/movie/bi/mi/point.naver?code=32240
https://movie.naver.com/movie/bi/mi/point.naver?code=31972
https://movie.naver.com/movie/bi/mi/point.naver?code=136667
https://movie.naver.com/movie/bi/mi/point.naver?code=30687
https://movie.naver.com/movie/bi/mi/point.naver?code=32223
https://movie.naver.com/movie/bi/mi/point.naver?code=31475
https://movie.naver.com/movie/bi/mi/point.naver?code=31312
https://movie.naver.com/movie/bi/mi/point.naver?code=31

https://movie.naver.com/movie/bi/mi/point.naver?code=31270
https://movie.naver.com/movie/bi/mi/point.naver?code=34566
https://movie.naver.com/movie/bi/mi/point.naver?code=33447
https://movie.naver.com/movie/bi/mi/point.naver?code=31160
https://movie.naver.com/movie/bi/mi/point.naver?code=32114
https://movie.naver.com/movie/bi/mi/point.naver?code=31239
https://movie.naver.com/movie/bi/mi/point.naver?code=33583
https://movie.naver.com/movie/bi/mi/point.naver?code=31572
https://movie.naver.com/movie/bi/mi/point.naver?code=35459
https://movie.naver.com/movie/bi/mi/point.naver?code=31788
https://movie.naver.com/movie/bi/mi/point.naver?code=31864
https://movie.naver.com/movie/bi/mi/point.naver?code=62257
https://movie.naver.com/movie/bi/mi/point.naver?code=32580
https://movie.naver.com/movie/bi/mi/point.naver?code=34581
https://movie.naver.com/movie/bi/mi/point.naver?code=38700
https://movie.naver.com/movie/bi/mi/point.naver?code=32108
https://movie.naver.com/movie/bi/mi/point.naver?code=342

https://movie.naver.com/movie/bi/mi/point.naver?code=33870
https://movie.naver.com/movie/bi/mi/point.naver?code=31797
https://movie.naver.com/movie/bi/mi/point.naver?code=38819
https://movie.naver.com/movie/bi/mi/point.naver?code=35162
https://movie.naver.com/movie/bi/mi/point.naver?code=34093
https://movie.naver.com/movie/bi/mi/point.naver?code=37100
https://movie.naver.com/movie/bi/mi/point.naver?code=35974
https://movie.naver.com/movie/bi/mi/point.naver?code=37143
https://movie.naver.com/movie/bi/mi/point.naver?code=19923
https://movie.naver.com/movie/bi/mi/point.naver?code=36982
https://movie.naver.com/movie/bi/mi/point.naver?code=33550
https://movie.naver.com/movie/bi/mi/point.naver?code=34225
https://movie.naver.com/movie/bi/mi/point.naver?code=33682
https://movie.naver.com/movie/bi/mi/point.naver?code=31905
https://movie.naver.com/movie/bi/mi/point.naver?code=34416
https://movie.naver.com/movie/bi/mi/point.naver?code=34198
https://movie.naver.com/movie/bi/mi/point.naver?code=359

...

https://movie.naver.com/movie/bi/mi/point.naver?code=63739
https://movie.naver.com/movie/bi/mi/point.naver?code=133216
https://movie.naver.com/movie/bi/mi/point.naver?code=52585
https://movie.naver.com/movie/bi/mi/point.naver?code=69528
https://movie.naver.com/movie/bi/mi/point.naver?code=66046
https://movie.naver.com/movie/bi/mi/point.naver?code=47381
https://movie.naver.com/movie/bi/mi/point.naver?code=53025
https://movie.naver.com/movie/bi/mi/point.naver?code=51123
https://movie.naver.com/movie/bi/mi/point.naver?code=52074
https://movie.naver.com/movie/bi/mi/point.naver?code=53724
https://movie.naver.com/movie/bi/mi/point.naver?code=70393
https://movie.naver.com/movie/bi/mi/point.naver?code=52957
https://movie.naver.com/movie/bi/mi/point.naver?code=54290
https://movie.naver.com/movie/bi/mi/point.naver?code=47493
https://movie.naver.com/movie/bi/mi/point.naver?code=45938
https://movie.naver.com/movie/bi/mi/point.naver?code=49228
https://movie.naver.com/movie/bi/mi/point.naver?code=70

https://movie.naver.com/movie/bi/mi/point.naver?code=102405
https://movie.naver.com/movie/bi/mi/point.naver?code=32619
https://movie.naver.com/movie/bi/mi/point.naver?code=74400
https://movie.naver.com/movie/bi/mi/point.naver?code=74235
https://movie.naver.com/movie/bi/mi/point.naver?code=77292
https://movie.naver.com/movie/bi/mi/point.naver?code=48211
https://movie.naver.com/movie/bi/mi/point.naver?code=84271
https://movie.naver.com/movie/bi/mi/point.naver?code=102857
https://movie.naver.com/movie/bi/mi/point.naver?code=52278
https://movie.naver.com/movie/bi/mi/point.naver?code=76695
https://movie.naver.com/movie/bi/mi/point.naver?code=73638
https://movie.naver.com/movie/bi/mi/point.naver?code=84292
https://movie.naver.com/movie/bi/mi/point.naver?code=71422
https://movie.naver.com/movie/bi/mi/point.naver?code=69542
https://movie.naver.com/movie/bi/mi/point.naver?code=70392
https://movie.naver.com/movie/bi/mi/point.naver?code=72443
https://movie.naver.com/movie/bi/mi/point.naver?code=7

https://movie.naver.com/movie/bi/mi/point.naver?code=76791
https://movie.naver.com/movie/bi/mi/point.naver?code=54948
https://movie.naver.com/movie/bi/mi/point.naver?code=69866
https://movie.naver.com/movie/bi/mi/point.naver?code=79104
https://movie.naver.com/movie/bi/mi/point.naver?code=77171
https://movie.naver.com/movie/bi/mi/point.naver?code=52958
https://movie.naver.com/movie/bi/mi/point.naver?code=71081
https://movie.naver.com/movie/bi/mi/point.naver?code=65132
https://movie.naver.com/movie/bi/mi/point.naver?code=67587
https://movie.naver.com/movie/bi/mi/point.naver?code=86859
https://movie.naver.com/movie/bi/mi/point.naver?code=52348
https://movie.naver.com/movie/bi/mi/point.naver?code=52530
https://movie.naver.com/movie/bi/mi/point.naver?code=73402
https://movie.naver.com/movie/bi/mi/point.naver?code=49593
https://movie.naver.com/movie/bi/mi/point.naver?code=51158
https://movie.naver.com/movie/bi/mi/point.naver?code=70564
https://movie.naver.com/movie/bi/mi/point.naver?code=779

https://movie.naver.com/movie/bi/mi/point.naver?code=77144
https://movie.naver.com/movie/bi/mi/point.naver?code=73296
https://movie.naver.com/movie/bi/mi/point.naver?code=77129
https://movie.naver.com/movie/bi/mi/point.naver?code=78927
https://movie.naver.com/movie/bi/mi/point.naver?code=52757
https://movie.naver.com/movie/bi/mi/point.naver?code=87201
https://movie.naver.com/movie/bi/mi/point.naver?code=70667
https://movie.naver.com/movie/bi/mi/point.naver?code=53969
https://movie.naver.com/movie/bi/mi/point.naver?code=52979
https://movie.naver.com/movie/bi/mi/point.naver?code=74601
https://movie.naver.com/movie/bi/mi/point.naver?code=133050
https://movie.naver.com/movie/bi/mi/point.naver?code=72007
https://movie.naver.com/movie/bi/mi/point.naver?code=71230
https://movie.naver.com/movie/bi/mi/point.naver?code=75744
https://movie.naver.com/movie/bi/mi/point.naver?code=71563
https://movie.naver.com/movie/bi/mi/point.naver?code=72960
https://movie.naver.com/movie/bi/mi/point.naver?code=78

https://movie.naver.com/movie/bi/mi/point.naver?code=76373
https://movie.naver.com/movie/bi/mi/point.naver?code=51700
https://movie.naver.com/movie/bi/mi/point.naver?code=72357
https://movie.naver.com/movie/bi/mi/point.naver?code=71498
https://movie.naver.com/movie/bi/mi/point.naver?code=71073
https://movie.naver.com/movie/bi/mi/point.naver?code=73787
https://movie.naver.com/movie/bi/mi/point.naver?code=76951
https://movie.naver.com/movie/bi/mi/point.naver?code=73376
https://movie.naver.com/movie/bi/mi/point.naver?code=72718
https://movie.naver.com/movie/bi/mi/point.naver?code=51022
https://movie.naver.com/movie/bi/mi/point.naver?code=49054
https://movie.naver.com/movie/bi/mi/point.naver?code=51923
https://movie.naver.com/movie/bi/mi/point.naver?code=76760
https://movie.naver.com/movie/bi/mi/point.naver?code=69958
https://movie.naver.com/movie/bi/mi/point.naver?code=67374
https://movie.naver.com/movie/bi/mi/point.naver?code=73193
https://movie.naver.com/movie/bi/mi/point.naver?code=660

In [10]:
crawling(2011, 2015)

https://movie.naver.com/movie/bi/mi/basic.naver?code=72327
https://movie.naver.com/movie/bi/mi/basic.naver?code=77990
https://movie.naver.com/movie/bi/mi/basic.naver?code=90307
https://movie.naver.com/movie/bi/mi/basic.naver?code=82680
https://movie.naver.com/movie/bi/mi/basic.naver?code=88285
https://movie.naver.com/movie/bi/mi/basic.naver?code=88286
https://movie.naver.com/movie/bi/mi/basic.naver?code=89772
https://movie.naver.com/movie/bi/mi/basic.naver?code=82568
https://movie.naver.com/movie/bi/mi/basic.naver?code=78846
https://movie.naver.com/movie/bi/mi/basic.naver?code=83511
https://movie.naver.com/movie/bi/mi/basic.naver?code=48246
https://movie.naver.com/movie/bi/mi/basic.naver?code=97892
https://movie.naver.com/movie/bi/mi/basic.naver?code=87049
https://movie.naver.com/movie/bi/mi/basic.naver?code=85611
https://movie.naver.com/movie/bi/mi/basic.naver?code=85143
https://movie.naver.com/movie/bi/mi/basic.naver?code=86334
https://movie.naver.com/movie/bi/mi/basic.naver?code=518

https://movie.naver.com/movie/bi/mi/basic.naver?code=81419
https://movie.naver.com/movie/bi/mi/basic.naver?code=83280
https://movie.naver.com/movie/bi/mi/basic.naver?code=89775
https://movie.naver.com/movie/bi/mi/basic.naver?code=95653
https://movie.naver.com/movie/bi/mi/basic.naver?code=72177
https://movie.naver.com/movie/bi/mi/basic.naver?code=101494
https://movie.naver.com/movie/bi/mi/basic.naver?code=84829
https://movie.naver.com/movie/bi/mi/basic.naver?code=84293
https://movie.naver.com/movie/bi/mi/basic.naver?code=78245
https://movie.naver.com/movie/bi/mi/basic.naver?code=72075
https://movie.naver.com/movie/bi/mi/basic.naver?code=91150
https://movie.naver.com/movie/bi/mi/basic.naver?code=73075
https://movie.naver.com/movie/bi/mi/basic.naver?code=76444
https://movie.naver.com/movie/bi/mi/basic.naver?code=101448
https://movie.naver.com/movie/bi/mi/basic.naver?code=90446
https://movie.naver.com/movie/bi/mi/basic.naver?code=95964
https://movie.naver.com/movie/bi/mi/basic.naver?code=5

https://movie.naver.com/movie/bi/mi/basic.naver?code=98087
https://movie.naver.com/movie/bi/mi/basic.naver?code=85094
https://movie.naver.com/movie/bi/mi/basic.naver?code=85984
https://movie.naver.com/movie/bi/mi/basic.naver?code=89220
https://movie.naver.com/movie/bi/mi/basic.naver?code=74528
https://movie.naver.com/movie/bi/mi/basic.naver?code=85095
https://movie.naver.com/movie/bi/mi/basic.naver?code=47495
https://movie.naver.com/movie/bi/mi/basic.naver?code=44959
https://movie.naver.com/movie/bi/mi/basic.naver?code=84020
https://movie.naver.com/movie/bi/mi/basic.naver?code=113578
https://movie.naver.com/movie/bi/mi/basic.naver?code=84319
https://movie.naver.com/movie/bi/mi/basic.naver?code=89229
https://movie.naver.com/movie/bi/mi/basic.naver?code=78300
https://movie.naver.com/movie/bi/mi/basic.naver?code=89285
https://movie.naver.com/movie/bi/mi/basic.naver?code=76966
https://movie.naver.com/movie/bi/mi/basic.naver?code=51702
https://movie.naver.com/movie/bi/mi/basic.naver?code=84

https://movie.naver.com/movie/bi/mi/basic.naver?code=79439
https://movie.naver.com/movie/bi/mi/basic.naver?code=71492
https://movie.naver.com/movie/bi/mi/basic.naver?code=81905
https://movie.naver.com/movie/bi/mi/basic.naver?code=86120
https://movie.naver.com/movie/bi/mi/basic.naver?code=75068
https://movie.naver.com/movie/bi/mi/basic.naver?code=86981
https://movie.naver.com/movie/bi/mi/basic.naver?code=91412
https://movie.naver.com/movie/bi/mi/basic.naver?code=103378
https://movie.naver.com/movie/bi/mi/basic.naver?code=77039
https://movie.naver.com/movie/bi/mi/basic.naver?code=83165
https://movie.naver.com/movie/bi/mi/basic.naver?code=101187
https://movie.naver.com/movie/bi/mi/basic.naver?code=50746
https://movie.naver.com/movie/bi/mi/basic.naver?code=88720
https://movie.naver.com/movie/bi/mi/basic.naver?code=70017
https://movie.naver.com/movie/bi/mi/basic.naver?code=83500
https://movie.naver.com/movie/bi/mi/basic.naver?code=74832
https://movie.naver.com/movie/bi/mi/basic.naver?code=9

https://movie.naver.com/movie/bi/mi/basic.naver?code=94868
https://movie.naver.com/movie/bi/mi/basic.naver?code=101482
https://movie.naver.com/movie/bi/mi/basic.naver?code=98193
https://movie.naver.com/movie/bi/mi/basic.naver?code=53040
https://movie.naver.com/movie/bi/mi/basic.naver?code=89298
https://movie.naver.com/movie/bi/mi/basic.naver?code=82454
https://movie.naver.com/movie/bi/mi/basic.naver?code=74003
https://movie.naver.com/movie/bi/mi/basic.naver?code=76689
https://movie.naver.com/movie/bi/mi/basic.naver?code=85858
https://movie.naver.com/movie/bi/mi/basic.naver?code=68437
https://movie.naver.com/movie/bi/mi/basic.naver?code=84239
https://movie.naver.com/movie/bi/mi/basic.naver?code=50546
https://movie.naver.com/movie/bi/mi/basic.naver?code=86130
https://movie.naver.com/movie/bi/mi/basic.naver?code=52429
https://movie.naver.com/movie/bi/mi/basic.naver?code=76288
https://movie.naver.com/movie/bi/mi/basic.naver?code=74660
https://movie.naver.com/movie/bi/mi/basic.naver?code=77

...

https://movie.naver.com/movie/bi/mi/basic.naver?code=137678
https://movie.naver.com/movie/bi/mi/basic.naver?code=123170
https://movie.naver.com/movie/bi/mi/basic.naver?code=139599
https://movie.naver.com/movie/bi/mi/basic.naver?code=128248
https://movie.naver.com/movie/bi/mi/basic.naver?code=144707
https://movie.naver.com/movie/bi/mi/basic.naver?code=141709
https://movie.naver.com/movie/bi/mi/basic.naver?code=138863
https://movie.naver.com/movie/bi/mi/basic.naver?code=144399
https://movie.naver.com/movie/bi/mi/basic.naver?code=143232
https://movie.naver.com/movie/bi/mi/basic.naver?code=138920
https://movie.naver.com/movie/bi/mi/basic.naver?code=122535
https://movie.naver.com/movie/bi/mi/basic.naver?code=127354
https://movie.naver.com/movie/bi/mi/basic.naver?code=141421
https://movie.naver.com/movie/bi/mi/basic.naver?code=127426
https://movie.naver.com/movie/bi/mi/basic.naver?code=136648
https://movie.naver.com/movie/bi/mi/basic.naver?code=47321
https://movie.naver.com/movie/bi/mi/basic

https://movie.naver.com/movie/bi/mi/basic.naver?code=135615
https://movie.naver.com/movie/bi/mi/basic.naver?code=65551
https://movie.naver.com/movie/bi/mi/basic.naver?code=134830
https://movie.naver.com/movie/bi/mi/basic.naver?code=142112
https://movie.naver.com/movie/bi/mi/basic.naver?code=135303
https://movie.naver.com/movie/bi/mi/basic.naver?code=118395
https://movie.naver.com/movie/bi/mi/basic.naver?code=136702
https://movie.naver.com/movie/bi/mi/basic.naver?code=133064
https://movie.naver.com/movie/bi/mi/basic.naver?code=102862
https://movie.naver.com/movie/bi/mi/basic.naver?code=124844
https://movie.naver.com/movie/bi/mi/basic.naver?code=145439
https://movie.naver.com/movie/bi/mi/basic.naver?code=38522
https://movie.naver.com/movie/bi/mi/basic.naver?code=142008
https://movie.naver.com/movie/bi/mi/basic.naver?code=145118
https://movie.naver.com/movie/bi/mi/basic.naver?code=133062
https://movie.naver.com/movie/bi/mi/basic.naver?code=133075
https://movie.naver.com/movie/bi/mi/basic.

https://movie.naver.com/movie/bi/mi/basic.naver?code=133127
https://movie.naver.com/movie/bi/mi/basic.naver?code=119331
https://movie.naver.com/movie/bi/mi/basic.naver?code=134132
https://movie.naver.com/movie/bi/mi/basic.naver?code=50953
https://movie.naver.com/movie/bi/mi/basic.naver?code=136662
https://movie.naver.com/movie/bi/mi/basic.naver?code=123389
https://movie.naver.com/movie/bi/mi/basic.naver?code=141687
https://movie.naver.com/movie/bi/mi/basic.naver?code=120125
https://movie.naver.com/movie/bi/mi/basic.naver?code=125446
https://movie.naver.com/movie/bi/mi/basic.naver?code=145178
https://movie.naver.com/movie/bi/mi/basic.naver?code=143299
https://movie.naver.com/movie/bi/mi/basic.naver?code=113323
https://movie.naver.com/movie/bi/mi/basic.naver?code=143820
https://movie.naver.com/movie/bi/mi/basic.naver?code=143829
https://movie.naver.com/movie/bi/mi/basic.naver?code=143231
https://movie.naver.com/movie/bi/mi/basic.naver?code=141726
https://movie.naver.com/movie/bi/mi/basic

https://movie.naver.com/movie/bi/mi/basic.naver?code=111990
https://movie.naver.com/movie/bi/mi/basic.naver?code=76243
https://movie.naver.com/movie/bi/mi/basic.naver?code=143505
https://movie.naver.com/movie/bi/mi/basic.naver?code=85818
https://movie.naver.com/movie/bi/mi/basic.naver?code=110423
https://movie.naver.com/movie/bi/mi/basic.naver?code=138244
https://movie.naver.com/movie/bi/mi/basic.naver?code=120543
https://movie.naver.com/movie/bi/mi/basic.naver?code=114249
https://movie.naver.com/movie/bi/mi/basic.naver?code=141480
https://movie.naver.com/movie/bi/mi/basic.naver?code=120103
https://movie.naver.com/movie/bi/mi/basic.naver?code=130341
https://movie.naver.com/movie/bi/mi/basic.naver?code=119175
https://movie.naver.com/movie/bi/mi/basic.naver?code=76594
https://movie.naver.com/movie/bi/mi/basic.naver?code=118347
https://movie.naver.com/movie/bi/mi/basic.naver?code=119968
https://movie.naver.com/movie/bi/mi/basic.naver?code=135601
https://movie.naver.com/movie/bi/mi/basic.n

https://movie.naver.com/movie/bi/mi/basic.naver?code=122647
https://movie.naver.com/movie/bi/mi/basic.naver?code=97695
https://movie.naver.com/movie/bi/mi/basic.naver?code=112268
https://movie.naver.com/movie/bi/mi/basic.naver?code=100643
https://movie.naver.com/movie/bi/mi/basic.naver?code=143838
https://movie.naver.com/movie/bi/mi/basic.naver?code=137264
https://movie.naver.com/movie/bi/mi/basic.naver?code=140136
https://movie.naver.com/movie/bi/mi/basic.naver?code=123470
https://movie.naver.com/movie/bi/mi/basic.naver?code=98149
https://movie.naver.com/movie/bi/mi/basic.naver?code=132976
https://movie.naver.com/movie/bi/mi/basic.naver?code=131418
https://movie.naver.com/movie/bi/mi/basic.naver?code=139675
https://movie.naver.com/movie/bi/mi/basic.naver?code=144428
https://movie.naver.com/movie/bi/mi/basic.naver?code=139106
https://movie.naver.com/movie/bi/mi/basic.naver?code=140245
https://movie.naver.com/movie/bi/mi/basic.naver?code=142623
https://movie.naver.com/movie/bi/mi/basic.

In [ ]:
crawling(2016, 2020)